In [225]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
#No warnings
import warnings
warnings.filterwarnings('ignore') # Filter out warnings

# data analysis and wrangling
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB # Gaussian Naive Bays
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb


In [210]:
data = pd.read_csv('regressiondatain.csv')
data

,Loan_ID,Age,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,NaN,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,NaN,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,NaN,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,NaN,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,NaN,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,NaN,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,NaN,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,NaN,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,NaN,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,NaN,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [211]:
data.fillna(method='ffill',inplace=True)
data.fillna(method='bfill',inplace=True)

In [212]:
#change dependents to int from 0-3
data['Dependents']=data['Dependents'].apply(lambda x: int(str(x)[:1]))

In [213]:
#married yes =1
data['Married']=data['Married'].apply(lambda x: 1 if x=='Yes' else 0)

In [214]:
data.drop(['Loan_ID','Age'],axis=1,inplace=True)

In [215]:
#loanstatus,self employed, Graduate?, gender 1=male
data['Loan_Status']=data['Loan_Status'].apply(lambda x: 1 if x=='Y' else 0)
data['Self_Employed']=data['Self_Employed'].apply(lambda x: 1 if x=='Yes' else 0)
data['Gender']=data['Gender'].apply(lambda x: 1 if x=='Male' else 0)

In [216]:
data['Graduate']=data['Education'].apply(lambda x: 0 if x=='Not Graduate' else 1)
data.drop('Education',axis=1,inplace=True)

In [217]:
# Title dummy mapping: Map titles to binary dummy columns

binary_encoded = pd.get_dummies(data.Property_Area)
data[binary_encoded.columns] = binary_encoded

In [218]:
data.drop('Property_Area',axis=1,inplace=True)

Setup Training and Validation Sets

In [223]:
#Final data set
data

,Gender,Married,Dependents,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Graduate,Rural,Semiurban,Urban
0,1,0,0,0,5849,0.0,128.0,360.0,1.0,1,1,0,0,1
1,1,1,1,0,4583,1508.0,128.0,360.0,1.0,0,1,1,0,0
2,1,1,0,1,3000,0.0,66.0,360.0,1.0,1,1,0,0,1
3,1,1,0,0,2583,2358.0,120.0,360.0,1.0,1,0,0,0,1
4,1,0,0,0,6000,0.0,141.0,360.0,1.0,1,1,0,0,1
5,1,1,2,1,5417,4196.0,267.0,360.0,1.0,1,1,0,0,1
6,1,1,0,0,2333,1516.0,95.0,360.0,1.0,1,0,0,0,1
7,1,1,3,0,3036,2504.0,158.0,360.0,0.0,0,1,0,1,0
8,1,1,2,0,4006,1526.0,168.0,360.0,1.0,1,1,0,0,1
9,1,1,1,0,12841,10968.0,349.0,360.0,1.0,0,1,0,1,0


In [219]:
X = data.drop("Loan_Status", axis=1) # Training & Validation data
Y = data["Loan_Status"]              # Response / Target Variable

print(X.shape, Y.shape)

(614, 13) (614,)


In [220]:
# Split training set so that we validate on 20% of the data
# Note that our algorithms will never have seen the validation 

np.random.seed(1337) # set random seed for reproducibility

from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = \
                train_test_split(X, Y, test_size=0.2)

print('Training Samples:', X_train.shape, Y_train.shape)
print('Validation Samples:', X_val.shape, Y_val.shape)

Training Samples: (491, 13) (491,)
Validation Samples: (123, 13) (123,)


# Logistic regression

In [221]:
logreg = LogisticRegression()           # create
logreg.fit(X_train, Y_train)            # train
acc_log_2 = logreg.score(X_val, Y_val)  # predict & evaluate

print('Logistic Regression accuracy:',\
      str(round(acc_log_2*100,2)),'%')

Logistic Regression accuracy: 82.11 %


KNN

In [226]:
knn = KNeighborsClassifier(n_neighbors = 5)                  # instantiate
knn.fit(X_train, Y_train)                                    # fit
acc_knn = knn.score(X_val, Y_val)                            # predict + evaluate

print('K-Nearest Neighbors labeling accuracy:', str(round(acc_knn*100,2)),'%')

K-Nearest Neighbors labeling accuracy: 66.67 %


# Support Vector Machine

In [227]:
# Support Vector Machines Classifier (non-linear kernel)
svc = SVC()                                                  # instantiate
svc.fit(X_train, Y_train)                                    # fit
acc_svc = svc.score(X_val, Y_val)                            # predict + evaluate

print('Support Vector Machines labeling accuracy:', str(round(acc_svc*100,2)),'%')

Support Vector Machines labeling accuracy: 72.36 %


# Perceptron

In [228]:
perceptron = Perceptron()                                    # instantiate 
perceptron.fit(X_train, Y_train)                             # fit
acc_perceptron = perceptron.score(X_val, Y_val)              # predict + evalaute

print('Perceptron labeling accuracy:', str(round(acc_perceptron*100,2)),'%')

Perceptron labeling accuracy: 47.97 %


# Gradient Boosting

In [229]:
# XGBoost, same API as scikit-learn
gradboost = xgb.XGBClassifier(n_estimators=1000)             # instantiate
gradboost.fit(X_train, Y_train)                              # fit
acc_xgboost = gradboost.score(X_val, Y_val)                  # predict + evalute

print('XGBoost labeling accuracy:', str(round(acc_xgboost*100,2)),'%')

XGBoost labeling accuracy: 74.8 %


# Random Forest

In [230]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=500)   # instantiate
random_forest.fit(X_train, Y_train)                         # fit
acc_rf = random_forest.score(X_val, Y_val)                  # predict + evaluate

print('Random Forest labeling accuracy:', str(round(acc_rf*100,2)),'%')

Random Forest labeling accuracy: 82.11 %


#  Neural Networks (Deep Learning)

In [231]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add( Dense(units=300, activation='relu', input_shape=(13,) ))
model.add( Dense(units=100, activation='relu'))
model.add( Dense(units=50, activation='relu'))
model.add( Dense(units=1, activation='sigmoid') )
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs = 50, batch_size= 50)
# # Evaluate the model Accuracy on test set
print('Neural Network accuracy:',str(round(model.evaluate(X_val, Y_val, batch_size=50,verbose=False)[1]*100,2)),'%')

Using TensorFlow backend.


Epoch 1/50
491/491 [==============================] - 0s 758us/step - loss: 10.9854 - acc: 0.3157
Epoch 2/50
491/491 [==============================] - 0s 41us/step - loss: 10.9314 - acc: 0.3218
Epoch 3/50
491/491 [==============================] - 0s 35us/step - loss: 10.9314 - acc: 0.3218
Epoch 4/50
491/491 [==============================] - 0s 45us/step - loss: 10.9314 - acc: 0.3218
Epoch 5/50
491/491 [==============================] - 0s 47us/step - loss: 10.9314 - acc: 0.3218
Epoch 6/50
491/491 [==============================] - 0s 42us/step - loss: 10.9314 - acc: 0.3218
Epoch 7/50
491/491 [==============================] - 0s 52us/step - loss: 10.9314 - acc: 0.3218
Epoch 8/50
491/491 [==============================] - 0s 41us/step - loss: 10.9314 - acc: 0.3218
Epoch 9/50
491/491 [==============================] - 0s 48us/step - loss: 10.9314 - acc: 0.3218
Epoch 10/50
491/491 [==============================] - 0s 51us/step - loss: 10.9314 - acc: 0.3218
Epoch 11/50
491/491 [=======